In [277]:
# df
import pandas as pd
import string
import numpy as np
import time

# visualize
import matplotlib.pyplot as plt
import seaborn as sns

# nlp libraries
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

# preprocessing tools
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# neural net's  and pipeline library
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [332]:
train=pd.read_csv(r'C:\Users\Ismat\OneDrive\Рабочий стол\document\etc\lesson_course\projects\nlp__kaggle_competition\train.csv')
test=pd.read_csv(r'C:\Users\Ismat\OneDrive\Рабочий стол\document\etc\lesson_course\projects\nlp__kaggle_competition\test.csv')
sample_submission=pd.read_csv(r'C:\Users\Ismat\OneDrive\Рабочий стол\document\etc\lesson_course\projects\nlp__kaggle_competition\sample_submission.csv')
sample_submission

,id,target
0,0,0
1,2,0
2,3,0
3,9,0
4,11,0
...,...,...
3258,10861,0
3259,10865,0
3260,10868,0
3261,10874,0


In [343]:
test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [333]:
def text_cleaner (text) :
    punctuation = [char for char in text if char not in string.punctuation]
    punctuation = ''.join (punctuation)
    return [word for word in punctuation.split() if word.lower() not in stopwords.words('english')]

In [334]:
%%time
transformer = CountVectorizer(analyzer=text_cleaner).fit(train['text'])

transformer_bow = transformer.transform(train['text'])

tf_idf_transformer = TfidfTransformer().fit(transformer_bow)

text_tfidf = tf_idf_transformer.transform(transformer_bow)

futures_train,futures_test,target_train,target_test = train_test_split(text_tfidf,train['target'],test_size=.2) 

CPU times: total: 25 s
Wall time: 26.2 s


In [335]:
%%time

ann = tf.keras.models.Sequential()

# early_stop = EarlyStopping(monitor='accuracy',mode='min')

ann.add(tf.keras.layers.Dense(units=30, activation='relu'))
ann.add(Dropout(.02))

ann.add(tf.keras.layers.Dense(units=25, activation='relu'))
ann.add(Dropout(.02))

ann.add(tf.keras.layers.Dense(units=20, activation='relu'))
ann.add(Dropout(.02))

ann.add(tf.keras.layers.Dense(units=25, activation='relu'))
ann.add(Dropout(.02))

ann.add(tf.keras.layers.Dense(units=30, activation='relu'))
ann.add(Dropout(.02))

ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

ann.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

CPU times: total: 0 ns
Wall time: 13 ms


In [353]:
ann.fit(x=futures_train.toarray(), 
        y=target_train, 
        batch_size = 251, 
        epochs =100
        #,callbacks=[early_stop]
       )       

Epoch 1/100
25/25 [==============================] - 1s 27ms/step - loss: 0.0056 - accuracy: 0.9969
Epoch 2/100
25/25 [==============================] - 1s 20ms/step - loss: 0.0056 - accuracy: 0.9966
Epoch 3/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0057 - accuracy: 0.9966
Epoch 4/100
25/25 [==============================] - 0s 12ms/step - loss: 0.0057 - accuracy: 0.9966
Epoch 5/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0059 - accuracy: 0.9966
Epoch 6/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0055 - accuracy: 0.9969
Epoch 7/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0066 - accuracy: 0.9962
Epoch 8/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0054 - accuracy: 0.9961
Epoch 9/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0061 - accuracy: 0.9964
Epoch 10/100
25/25 [==============================] - 0s 11ms/step - loss: 0.0058 - accuracy: 0.9962

In [354]:
ann_prediction = ann.predict(x=futures_test.toarray())
ann_prediction = (ann_prediction > 0.5)
ann_prediction

48/48 [==============================] - 0s 2ms/step


array([[ True],
       [False],
       [False],
       ...,
       [ True],
       [False],
       [False]])

In [355]:
print (classification_report(target_test, ann_prediction))

              precision    recall  f1-score   support

           0       0.77      0.60      0.68       848
           1       0.61      0.78      0.68       675

    accuracy                           0.68      1523
   macro avg       0.69      0.69      0.68      1523
weighted avg       0.70      0.68      0.68      1523



In [339]:
# ann_dff=pd.DataFrame(ann.history.history)
# print(ann_dff)
# ann_dff.plot()

In [356]:
%%time
transformer_1 = CountVectorizer(analyzer=text_cleaner).fit(test['text'])

transformer_bow_1 = transformer_1.transform(test['text'])

tf_idf_transformer_1 = TfidfTransformer().fit(transformer_bow_1)

text_tfidf_1 = tf_idf_transformer_1.transform(transformer_bow_1)


CPU times: total: 10.7 s
Wall time: 11.2 s


In [358]:
# test_prediction = ann.predict(x=text_tfidf_1.toarray())